In [2]:
!python -V

Python 3.12.11


In [3]:
import pandas as pd

In [4]:
import pickle

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [7]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2025/07/23 12:02:33 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/07/23 12:02:33 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<Experiment: artifact_location='/workspaces/ai-infra-journey/mlops-training/mlruns/1', creation_time=1753266959502, experiment_id='1', last_update_time=1753266959502, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [8]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [9]:
df_sample = pd.read_parquet('./data/green_tripdata_2025-01.parquet')
df_sample.dtypes

VendorID                          int32
lpep_pickup_datetime     datetime64[us]
lpep_dropoff_datetime    datetime64[us]
store_and_fwd_flag               object
RatecodeID                      float64
PULocationID                      int32
DOLocationID                      int32
passenger_count                 float64
trip_distance                   float64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
ehail_fee                       float64
improvement_surcharge           float64
total_amount                    float64
payment_type                    float64
trip_type                       float64
congestion_surcharge            float64
cbd_congestion_fee              float64
dtype: object

In [10]:
df_train = read_dataframe('./data/green_tripdata_2025-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2025-02.parquet')

In [11]:
df_train.dtypes

VendorID                          int32
lpep_pickup_datetime     datetime64[us]
lpep_dropoff_datetime    datetime64[us]
store_and_fwd_flag               object
RatecodeID                      float64
PULocationID                     object
DOLocationID                     object
passenger_count                 float64
trip_distance                   float64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
ehail_fee                       float64
improvement_surcharge           float64
total_amount                    float64
payment_type                    float64
trip_type                       float64
congestion_surcharge            float64
cbd_congestion_fee              float64
duration                        float64
dtype: object

In [12]:
len(df_train), len(df_val)

(46307, 44218)

In [13]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [14]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [15]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [16]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred)

37.19629658380478

In [17]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [18]:
with mlflow.start_run():

    mlflow.set_tag("developer", "cristian")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [19]:
import xgboost as xgb

In [20]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.12/site-packages/hyperopt/atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [21]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [22]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [23]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [12:02:36] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:8.54239                           
[1]	validation-rmse:8.05524                           
[2]	validation-rmse:7.63739                           
[3]	validation-rmse:7.28044                           
[4]	validation-rmse:6.97577                           
[5]	validation-rmse:6.71820                           
[6]	validation-rmse:6.49992                           
[7]	validation-rmse:6.31736                           
[8]	validation-rmse:6.16367                           
[9]	validation-rmse:6.03462                           
[10]	validation-rmse:5.92636                          
[11]	validation-rmse:5.83684                          
[12]	validation-rmse:5.76148                          
[13]	validation-rmse:5.69905                          
[14]	validation-rmse:5.64654                          
[15]	validation-rmse:5.60341                          
[16]	validation-rmse:5.56584                          
[17]	validation-rmse:5.53522                          
[18]	valid

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [12:03:37] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:5.94527                                                     
[1]	validation-rmse:5.57952                                                     
[2]	validation-rmse:5.50564                                                     
[3]	validation-rmse:5.48484                                                     
[4]	validation-rmse:5.47778                                                     
[5]	validation-rmse:5.46762                                                     
[6]	validation-rmse:5.44529                                                     
[7]	validation-rmse:5.43668                                                     
[8]	validation-rmse:5.43057                                                     
[9]	validation-rmse:5.42842                                                     
[10]	validation-rmse:5.42729                                                    
[11]	validation-rmse:5.42233                                                    
[12]	validation-rmse:5.42124

...

[0]	validation-rmse:5.75498                                                      
[1]	validation-rmse:5.58055                                                      
[2]	validation-rmse:5.57356                                                      
[3]	validation-rmse:5.57065                                                      
[4]	validation-rmse:5.55884                                                      
[5]	validation-rmse:5.55850                                                      
[6]	validation-rmse:5.55064                                                      
[7]	validation-rmse:5.55412                                                      
[8]	validation-rmse:5.54219                                                      
[9]	validation-rmse:5.54292                                                      
[10]	validation-rmse:5.54297                                                     
[11]	validation-rmse:5.54526                                                     
[12]	validation-

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [12:25:58] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:8.22835                                                      
[1]	validation-rmse:7.53908                                                      
[2]	validation-rmse:7.00722                                                      
[3]	validation-rmse:6.59668                                                      
[4]	validation-rmse:6.28883                                                      
[5]	validation-rmse:6.05546                                                      
[6]	validation-rmse:5.87785                                                      
[7]	validation-rmse:5.74966                                                      
[8]	validation-rmse:5.65002                                                      
[9]	validation-rmse:5.58115                                                      
[10]	validation-rmse:5.52802                                                     
[11]	validation-rmse:5.48583                                                     
[12]	validation-

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [12:26:21] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:6.21220                                                      
[1]	validation-rmse:5.58431                                                      
[2]	validation-rmse:5.45106                                                      
[3]	validation-rmse:5.41573                                                      
[4]	validation-rmse:5.39142                                                      
[5]	validation-rmse:5.38298                                                      
[6]	validation-rmse:5.37776                                                      
[7]	validation-rmse:5.37356                                                      
[8]	validation-rmse:5.37124                                                      
[9]	validation-rmse:5.36899                                                      
[10]	validation-rmse:5.36021                                                     
[11]	validation-rmse:5.36117                                                     
[12]	validation-

In [24]:
mlflow.xgboost.autolog(disable=True)

In [ ]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [12:26:32] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[0]	validation-rmse:8.56745
[1]	validation-rmse:8.10033
[2]	validation-rmse:7.69739
[3]	validation-rmse:7.35105
[4]	validation-rmse:7.05239
[5]	validation-rmse:6.80020
[6]	validation-rmse:6.58162
[7]	validation-rmse:6.39726
[8]	validation-rmse:6.24197
[9]	validation-rmse:6.11059
[10]	validation-rmse:6.00193
[11]	validation-rmse:5.90700
[12]	validation-rmse:5.82752
[13]	validation-rmse:5.76123
[14]	validation-rmse:5.70306
[15]	validation-rmse:5.65560
[16]	validation-rmse:5.61619
[17]	validation-rmse:5.58065
[18]	validation-rmse:5.55157
[19]	validation-rmse:5.52734
[20]	validation-rmse:5.50858
[21]	validation-rmse:5.48872
[22]	validation-rmse:5.47379
[23]	validation-rmse:5.45957
[24]	validation-rmse:5.44705
[25]	validation-rmse:5.43706
[26]	validation-rmse:5.42713
[27]	validation-rmse:5.41891
[28]	validation-rmse:5.41283
[29]	validation-rmse:5.40819
[30]	validation-rmse:5.40244
[31]	validation-rmse:5.39854
[32]	validation-rmse:5.39446
[33]	validation-rmse:5.39004
[34]	validation-rmse:5.3

[103]	validation-rmse:5.34937
[104]	validation-rmse:5.34942
[105]	validation-rmse:5.34926
[106]	validation-rmse:5.34906
[107]	validation-rmse:5.34844
[108]	validation-rmse:5.34842
[109]	validation-rmse:5.34862
[110]	validation-rmse:5.34826
[111]	validation-rmse:5.34813
[112]	validation-rmse:5.34771
[113]	validation-rmse:5.34770
[114]	validation-rmse:5.34757
[115]	validation-rmse:5.34715
[116]	validation-rmse:5.34679
[117]	validation-rmse:5.34671
[118]	validation-rmse:5.34628
[119]	validation-rmse:5.34616
[120]	validation-rmse:5.34638
[121]	validation-rmse:5.34591
[122]	validation-rmse:5.34571
[123]	validation-rmse:5.34564
[124]	validation-rmse:5.34567
[125]	validation-rmse:5.34552
[126]	validation-rmse:5.34560
[127]	validation-rmse:5.34526
[128]	validation-rmse:5.34533
[129]	validation-rmse:5.34547
[130]	validation-rmse:5.34589
[131]	validation-rmse:5.34586
[132]	validation-rmse:5.34585
[133]	validation-rmse:5.34582
[134]	validation-rmse:5.34554
[135]	validation-rmse:5.34515
[136]	vali

2025/07/23 12:27:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.12/site-packages/mlflow/xgboost/__init__.py:168: UserWarning: [12:27:13] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/07/23 12:27:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


: 

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)
        